In [2]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn import linear_model
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

In [4]:
df = pd.read_csv('D:/finance-vix.txt',
                     sep=',',
                     parse_dates=[0],
                     infer_datetime_format=True,
                     index_col=False,
                     na_values=['nan', '?'])

#Fill na values with 0
df.fillna(0, inplace=True)

#Drop date from dataset
df = df.drop('Date', axis=1)
display(df.head())

,VIX Open,VIX High,VIX Low,VIX Close
0,17.96,18.68,17.54,18.22
1,18.45,18.49,17.44,17.49
2,17.66,17.67,16.19,16.73
3,16.72,16.75,15.50,15.50
4,15.42,15.68,15.32,15.61


In [5]:
#To change the name of VIX Open column to OPEN 
df = df.rename(columns={'VIX Open':'Open'})
display(df.describe())

,Open,VIX High,VIX Low,VIX Close
count,3725.000000,3725.000000,3725.000000,3725.000000
mean,18.457289,19.301055,17.611780,18.357952
std,8.879643,9.465319,8.269978,8.844818
min,9.010000,9.310000,8.560000,9.140000
25%,13.010000,13.470000,12.540000,12.910000
50%,15.720000,16.440000,15.070000,15.600000
75%,20.900000,21.910000,19.890000,20.790000
max,80.740000,89.530000,72.760000,80.860000


In [6]:
#Here we need to specify features and labels
X = df.drop('Open', axis=1)
Y = df['Open']

In [7]:
#Now split our data into train and test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=20)

print('X_train shape {}'.format(X_train.shape))
print('Y_train shape {}'.format(Y_train.shape))
print('X_test shape {}'.format(X_test.shape))
print('Y_test shape {}'.format(Y_test.shape))

X_train shape (2980, 3)
Y_train shape (2980,)
X_test shape (745, 3)
Y_test shape (745,)


In [8]:
#Scale data, otherwise model will fail
scaler = StandardScaler()
scaler = scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Set Decision Tree

In [10]:
#Create an instance
model = DecisionTreeRegressor()
#Fit the DT model
model.fit(X_train_scaled, Y_train)
#Predict the model
y_pred = model.predict(X_test_scaled)

DT_mse = mean_squared_error(Y_test, y_pred)
DT_mae = mean_absolute_error(Y_test, y_pred)

print('Mean Squared Error from Decision Tree: {}'.format(DT_mse))
print('Mean Absolute Error from Decision Tree: {}'.format(DT_mae))

Mean Squared Error from Decision Tree: 1.3091687248322148
Mean Absolute Error from Decision Tree: 0.5928456375838927


# Set Random Forest

In [11]:
model = RandomForestRegressor(n_estimators=30, random_state=30)
model.fit(X_train_scaled, Y_train)
y_pred = model.predict(X_test_scaled)
RF_mse = mean_squared_error(Y_test, y_pred)
RF_mae = mean_absolute_error(Y_test, y_pred)

print('Mean squared error using Random Forest: {}'.format(RF_mse))
print('Mean absolute error Using Random Forest: {}'.format(RF_mae))

Mean squared error using Random Forest: 0.6342885886651759
Mean absolute error Using Random Forest: 0.4713964205816558


### To investigate which parameters are most importance in random forest decision

In [12]:
features = list(X.columns)
important_features = pd.Series(model.feature_importances_, index=features).sort_values(ascending=False)
print(important_features)

VIX Low      0.838318
VIX High     0.156600
VIX Close    0.005082
dtype: float64
